# Analyse Rotation Spectra - Daily Medians

## Load Libraries

In [124]:
from andbro__querrySeismoData import __querrySeismoData
from obspy import UTCDateTime
from scipy.signal import welch
from numpy import log10, zeros, pi, append, linspace, mean, median, array, where, transpose, shape, histogram, arange
from numpy import logspace, linspace, log, log10, isinf, ones, nan, count_nonzero, sqrt, isnan
from pandas import DataFrame, concat, Series, date_range, read_csv, read_pickle
from tqdm import tqdm_notebook
from pathlib import Path
from scipy.stats import median_absolute_deviation as mad

import pickle
import matplotlib.pyplot as plt

In [125]:
def __get_minimal_psd(psds):

    from numpy import nanmin, array, nonzero
    
    min_psd = zeros(psds.shape[1])
    
    for f in range(psds.shape[1]):
        a = psds[:,f]
        min_psd[f] = nanmin(a[nonzero(a)])
    
    return min_psd

In [126]:
def __get_median_psd(psds):

    from numpy import median, zeros, isnan

    med_psd = zeros(psds.shape[1])

    for f in range(psds.shape[1]):
        a = psds[:,f]
        med_psd[f] = median(a[~isnan(a)])

    return med_psd

In [127]:
def __get_minimum_psd(psds, f_tmp):
    
    from numpy import median
    
    f_before = f_tmp[0]
    f_tmp[0] = 1e-9
    
    for i, psd in enumerate(psds):
        if i == 0: 
            lowest_value = (psds[0]/f_tmp).sum()
#             lowest_value = median(psds[0]/f_tmp)
            idx = 0

        value = (psd/f_tmp).sum() 
#         value = median(psd/f_tmp)
        
        if value < lowest_value and value != 0:
            lowest_value = value
            idx = i

    f_tmp[0] = f_before      
    
    return psds[idx]

In [128]:
def __get_array_from_dataframe(df):
    
    from pandas import to_numeric
    
    ## convert to list
    psds = []
    for col in array(df.columns):
        
        ## turn non-float series to float objects
        df[col] = to_numeric(df[col], errors = 'coerce')

        ## add to psds list
        psds.append(array(df[col]))

    return array(psds)

In [129]:
def __remove_noisy_psds(df, threshold_mean=1e-13):

    from numpy import delete
    
    l1 = len(df.columns)
    for col in df.columns:
#         print(col, type(col))
        if df[col].astype(float).mean() > threshold_mean:
            df = df.drop(columns=col)
    l2 = len(df.columns)
    print(f" -> removed {l1-l2} columns due to mean thresholds!")
    print(f" -> {l2} psds remain")
    
    return df

In [130]:
!ls "/import/kilauea-data/ROMY/RotationSpectra/data"

overview		     run_2019_RGRF-Z_3600_run6
run_2019_FUR-Z_3600	     run_2019_RGRF-Z_3600_run7
run_2019_FUR-Z_3600_testday  run_2019_RGRF-Z_3600_run8
run_2019_RGRF-_3600_GRA2     run_2019_RGRF-Z_3600_run9
run_2019_RGRF-E_3600_GRA     run_2019_RLAS-Z_3600
run_2019_RGRF-E_3600_GRA1    run_2019_RLAS-Z_3600_hours
run_2019_RGRF-E_3600_GRA2    run_2019_RLAS-Z_3600_testday
run_2019_RGRF-E_3600_GRB     run_2019_ROMY_ADR-E_3600
run_2019_RGRF-E_3600_GRB1    run_2019_ROMY_ADR-E_3600_all
run_2019_RGRF-E_3600_GRB2    run_2019_ROMY_ADR-N_3600_all
run_2019_RGRF-E_3600_GRC     run_2019_ROMY_ADR-Z_3600
run_2019_RGRF-E_3600_GRC1    run_2019_ROMY_ADR-Z_3600_all
run_2019_RGRF-E_3600_GRC2    run_2019_ROMY_ADR-Z_3600_classic
run_2019_RGRF-E_3600_GRF     run_2019_ROMY-E_3600
run_2019_RGRF-E_3600_GRF1    run_2019_ROMY-U_3600_hours
run_2019_RGRF-E_3600_GRF2    run_2019_ROMY-V_3600_hours
run_2019_RGRF-E_3600_GRF3    run_2019_ROMY-W_3600_hours
run_2019_RGRF-N_3600_GRA     run_2019_ROMY-Z_3600
run_2019_RGRF-N_3600_G

## Configurations

In [143]:
inname = "run_2022_ROMY-Z_3600"

inpath = f"/import/kilauea-data/ROMY/RotationSpectra/data/{inname}/"

config = pickle.load(open(inpath+inname+"_config.pkl", 'rb'))

config['save'] = True

config['inname'] = inname
config['inpath'] = inpath

config['frequency_limits'] = [0.0001, 1]

## Looping 

In [144]:
## load set of minimum psds
file = f"{config['inpath']}{config['inname']}_daily_medians.pkl"

print(f" -> loading \n {file} ...")
psds_all = read_pickle(file)
psds_all

## load frequencies
ff = pickle.load(open(f"{config['inpath']}{config['inname']}_frequency_axis.pkl", 'rb'))

 -> loading 
 /import/kilauea-data/ROMY/RotationSpectra/data/run_2022_ROMY-Z_3600/run_2022_ROMY-Z_3600_daily_medians.pkl ...


,20220101,20220102,20220103,20220104,20220105,20220106,20220107,20220108,20220109,20220110,...,20220522,20220523,20220524,20220525,20220526,20220527,20220528,20220529,20220530,20220531
0,2.394419e-21,9.215003e-22,1.108598e-21,7.531019e-22,1.220117e-21,6.596119e-21,2.150977e-20,2.160698e-20,3.663315e-21,3.359043e-21,...,5.154551e-13,7.326741e-13,1.297110e-12,1.027930e-12,7.447742e-13,1.070180e-12,7.101094e-13,7.758331e-13,1.639271e-12,1.240473e-12
1,2.759153e-20,8.410214e-21,7.172813e-21,8.047408e-21,2.832457e-20,7.065505e-20,2.676765e-19,1.009538e-19,2.201122e-20,7.698279e-20,...,3.095267e-12,3.931575e-12,7.039662e-12,6.289962e-12,5.261400e-12,4.370181e-12,3.831208e-12,4.432602e-12,9.876730e-12,1.021888e-11
2,6.805299e-21,3.930551e-21,5.544288e-21,2.991680e-21,5.688741e-21,1.139379e-20,3.462117e-20,1.428607e-20,5.978969e-21,5.410635e-21,...,1.895509e-12,1.466227e-12,3.254271e-12,2.967154e-12,3.136571e-12,2.657680e-12,2.125935e-12,2.570565e-12,5.413135e-12,5.408412e-12
3,4.865927e-21,2.997533e-21,7.051990e-21,2.115150e-21,3.112280e-21,5.230590e-21,2.333727e-20,1.188253e-20,7.681471e-21,6.355096e-21,...,1.048118e-12,7.718145e-13,1.056595e-12,1.319589e-12,1.290500e-12,1.161041e-12,1.049064e-12,1.391353e-12,2.095715e-12,2.455719e-12
4,2.226270e-21,2.665129e-21,9.358654e-21,3.409657e-21,2.846592e-21,4.009396e-21,1.219991e-20,8.776069e-21,1.060858e-20,4.038673e-21,...,5.422069e-13,6.569212e-13,1.131917e-12,8.779020e-13,8.784240e-13,9.078544e-13,6.309659e-13,1.000275e-12,1.776413e-12,1.770538e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12795,8.214185e-28,9.697313e-28,3.602713e-27,4.077091e-27,2.951422e-27,1.416817e-27,3.348682e-27,2.137889e-27,1.018163e-27,5.647295e-27,...,3.229133e-22,5.204264e-22,4.192141e-22,5.225426e-22,4.190005e-22,4.083937e-22,5.148939e-22,3.014657e-22,5.217459e-22,5.624631e-22
12796,9.338100e-28,1.211786e-27,4.244478e-27,4.080652e-27,3.180607e-27,6.101855e-28,3.347776e-27,2.003981e-27,1.086321e-27,5.238077e-27,...,3.634342e-22,4.007651e-22,4.669857e-22,5.421704e-22,4.837456e-22,3.499511e-22,4.422223e-22,4.184016e-22,4.246247e-22,5.265708e-22
12797,8.091853e-28,1.134491e-27,3.219472e-27,4.945395e-27,3.273453e-27,6.872440e-28,3.009282e-27,2.185313e-27,1.087860e-27,4.642356e-27,...,5.128463e-22,4.175307e-22,4.062912e-22,4.665435e-22,3.636375e-22,4.240603e-22,4.475626e-22,4.047060e-22,3.946237e-22,4.629828e-22
12798,7.183150e-28,1.207334e-27,2.380290e-27,3.242941e-27,2.762460e-27,8.099285e-28,2.794541e-27,2.376245e-27,9.539348e-28,4.357042e-27,...,3.942416e-22,5.336735e-22,2.822439e-22,3.500963e-22,4.440745e-22,4.668223e-22,4.268119e-22,3.823440e-22,3.912022e-22,3.684893e-22


In [145]:
if "RLAS-Z" in config['inname'].split("_"):
    psds_all = __remove_noisy_psds(psds_all, threshold_mean=1e-20)
elif "ROMY-Z" in config['inname'].split("_"):
    psds_all = __remove_noisy_psds(psds_all, threshold_mean=1e-18)
elif "ROMY-V" in config['inname'].split("_"):
    psds_all = __remove_noisy_psds(psds_all, threshold_mean=1e-18)
    

 -> removed 115 columns due to mean thresholds!
 -> 36 psds remain


Convert dataframes to array / matrix

In [84]:
# dates = ['20220314','20220315','20220316','20220317','20220318','20220319','20220320','20220321','20220322','20220323']
# psds_all = psds_all[dates]

In [90]:
dates = ["20190325", "20190326", "20190327"]

dates_new = []
for date in dates: 
    if date in psds_all.keys():
        dates_new.append(date)
psds_all = psds_all[dates_new]
psds_all

""


In [86]:
psds = __get_array_from_dataframe(psds_all)

columns = psds_all.columns

Cut to frequency axis and matrices to frequency limits as specified in config

In [87]:
ind = []
for i,f in enumerate(ff):
    if f >= config['frequency_limits'][0] and f <= config['frequency_limits'][1]:
        ind.append(i)

ff = ff[ind[0]:ind[-1]]
psds = psds[:,ind[0]:ind[-1]]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## Plotting

In [88]:
def __makeplot_colorlines(config, ff, psds, columns):

    from numpy import isnan, median, mean, std, array
    from scipy.stats import median_abs_deviation as mad
    
    psds_median, psds_mean, psds_std, psds_mad = [],[],[],[]
    for fcross in range(psds.shape[1]):
        data = psds[:,fcross]
        psds_median.append(median(data[~isnan(data)]))
#         psds_mean.append(mean(data[~isnan(data)]))
#         psds_std.append(std(data[~isnan(data)]))
#         psds_mad.append(mad(data[~isnan(data)]))
        
    psds_minimal = __get_minimal_psd(array(psds))
#     psds_minimum = __get_minimum_psd(array(psds), ff)
    
        
    ff = 1/ff
        
    ##____________________________
    
    fig, ax = plt.subplots(1,1, figsize=(15,10), sharey=False, sharex=True)

    font = 12

    N = psds.shape[0]
    colors = plt.cm.rainbow(linspace(0, 1, N))


    for n, psd in enumerate(psds):
        ax.loglog(ff, psd, color=colors[n], alpha=0.7)


    ## add scatter for colorbar object only
    for n, psd in enumerate(psds):
        p2 = ax.scatter(ff[0], psd[0], s=0., c=n, cmap='rainbow', vmin=0, vmax=N)


    ax.loglog(ff, psds_median, 'black', zorder=3, alpha=0.9)
    ax.loglog(ff, psds_minimal, 'grey', zorder=2, alpha=1.0)
    

#     ax.set_title("Minimal PSD", fontsize=font)
#     ax.set_title("Minimum PSD", fontsize=font)

    ax.set_xlabel("Period (s)", fontsize=font)

    ax.set_ylabel(r"PSD (rad$^2$/s$^2$/$Hz)$", fontsize=font)
    
    ax.grid(True, which="both", ls="-", alpha=0.5)
    
    ax.set_xlim(min(ff), max(ff))
#     ax.set_xlim(10**log10(config['frequency_limits'][0]), 10**log10(config['frequency_limits'][1]))
    
    ## set colorbar at bottom
    cbar = fig.colorbar(p2, orientation='horizontal', ax=ax, aspect=50)

    ## redefine ticks of colorbar
#     new_ticks=[]
#     for n, t in enumerate(cbar.get_ticks()):
#         print(n)
#         new_ticks.append(columns[int(t)])
#     print(new_ticks)
#     cbar.ax.set_xticklabels(columns)


    plt.show();
    
    out = DataFrame()
    out['frequencies'] = ff
    out['psds_minimal'] = array(psds_minimal)
    out['psds_median'] = array(psds_median)
#     out['psds_mean'] = array(psds_mean)
#     out['psds_std'] = array(psds_std)
#     out['psds_mad'] = array(psds_mad)
    
    
    return fig, out

In [89]:
fig, out = __makeplot_colorlines(config, ff, psds, columns)

IndexError: tuple index out of range

### Write to File

In [40]:
if config['save']:
    out.to_pickle(f"{inpath}{inname}_stats.pkl")
    print(f" -> writing to {inpath}{inname}_stats.pkl")

 -> writing to /import/kilauea-data/ROMY/RotationSpectra/data/run_2022_ROMY-Z_3600/run_2022_ROMY-Z_3600_stats.pkl


## Plot as DeciBel

In [44]:
def __conversion(data, informat=None, outformat=None):
    '''
    data:                  array 
    informat/outformat:    ASD, PSD, DB
    '''
    from numpy import log10, sqrt
    
    
    if informat == 'ASD' and outformat == 'PSD':
        return data**2
    
    elif informat == 'PSD' and outformat == 'ASD':
        return sqrt(data)

    elif informat == 'ASD' and outformat == 'DB':
        return 10*log10(data)
    
    elif informat == 'PSD' and outformat == 'DB':
        return 10*log10(data)
    
    elif informat == 'DB' and outformat == 'PSD':
        return 10**(data/10)
    
    elif informat == 'DB' and outformat == 'ASD':
        return sqrt(10**(data/10))
    
    else:
        print(" -> specified case not defined!")

In [45]:
psds_db = __conversion(psds, informat="PSD", outformat="DB")

In [46]:
def __makeplot_colorlines_decibel(config, ff, psds, columns):

    from numpy import isnan, median, mean, std, array
    from scipy.stats import median_abs_deviation as mad
    
    psds_median, psds_mean, psds_std, psds_mad = [],[],[],[]
    for fcross in range(psds.shape[1]):
        data = psds[:,fcross]
        psds_median.append(median(data[~isnan(data)]))
        psds_mean.append(mean(data[~isnan(data)]))
        psds_std.append(std(data[~isnan(data)]))
        psds_mad.append(mad(data[~isnan(data)]))
    
    psds_minimal = __get_minimal_psd(array(psds))
#     psds_minimum = __get_minimum_psd(array(psds), ff)
    
    
    ##____________________________
    
    fig, ax = plt.subplots(1,1, figsize=(15,10), sharey=False, sharex=True)

    font = 12

    N = psds.shape[0]
    colors = plt.cm.rainbow(linspace(0, 1, N))


    for n, psd in enumerate(psds):
        ax.semilogx(ff, psd, color=colors[n], alpha=0.7)

    ## add scatter for colorbar object only
    for n, psd in enumerate(psds):
        p2 = ax.scatter(ff[0], psd[0], s=0., c=n, cmap='rainbow', vmin=0, vmax=N)

    ax.semilogx(ff, psds_median, 'black', zorder=3, alpha=0.9)
    ax.semilogx(ff, array(psds_median)+array(psds_mad), 'grey', zorder=3, alpha=0.8)  
    ax.semilogx(ff, array(psds_median)-array(psds_mad), 'grey', zorder=3, alpha=0.8)  

#     ax.semilogx(ff, psds_minimal, 'black', zorder=2, alpha=1.0)
    

    ax.set_xlabel("Frequency (Hz)", fontsize=font)

    ax.set_ylabel(r"PSD (dB rel. to 1 $rad^2$/($s^2$ $Hz$)", fontsize=font)
  
    ax.grid(True, which="both", ls="-", alpha=0.5)

    
    ax.set_xlim(min(ff), max(ff))
#     ax.set_xlim(10**log10(config['frequency_limits'][0]), 10**log10(config['frequency_limits'][1]))
    
    ## set colorbar at bottom
    cbar = fig.colorbar(p2, orientation='horizontal', ax=ax, aspect=50)

    ## redefine ticks of colorbar
    dates = columns
    new_ticks=[]
    for t in cbar.get_ticks(): 
        new_ticks.append(dates.astype(str)[int(t)])
    cbar.ax.set_xticklabels(new_ticks)


    plt.show();
    
    out = DataFrame()
    out['frequencies'] = ff
    out['psds_minimal'] = array(psds_minimal)
    out['psds_median'] = array(psds_median)
    out['psds_mean'] = array(psds_mean)
    out['psds_std'] = array(psds_std)
    out['psds_mad'] = array(psds_mad)
    
    return fig, out

In [47]:
# fig, out_db = __makeplot_colorlines_decibel(config, ff, psds_db, columns)

### Write to file

In [25]:
if config['save']:
    out_db.to_csv(f"{inpath}{inname}_DB.csv", sep=",")
    print(f" -> writing to {inpath}{inname}_DB.csv")

NameError: name 'out_db' is not defined

## Calculate and Plot Density

Convert to logaritmic values

In [ ]:
# def __convert_to_log(psds):

#     print(psds.shape)

#     psds_new = zeros(psds.shape)
#     rows_with_zeros = 0
#     for i, psd in enumerate(psds):
#         if count_nonzero(psd) != len(psd):
#             rows_with_zeros +=1
#             psd = [nan for val in psd if val == 0]
#         psds_new[i,:] = log10(psd)
#         if isinf(psds_new[i,:]).any():
#             psds_new[i,:] = nan * ones(len(psds_new[i,:]))
            
#     return psds_new

# psds_log = __convert_to_log(psds)

In [ ]:
def __calculate_density(psd_array, bins=20, density=False):

    from numpy import argmax, std, median, isnan, array, histogram, nan, zeros, count_nonzero, log10, isinf
    from scipy.stats import median_abs_deviation as mad
    
    def __convert_to_log(in_psds):

        print(in_psds.shape)

        out_psds = zeros(in_psds.shape)
        rows_with_zeros = 0
        for i, psd in enumerate(in_psds):
            if count_nonzero(psd) != len(psd):
                rows_with_zeros +=1
                psd = [nan for val in psd if val == 0]
            out_psds[i,:] = log10(psd)
            if isinf(out_psds[i,:]).any():
                out_psds[i,:] = nan * ones(len(out_psds[i,:]))

        return out_psds

    psd_array = __convert_to_log(psd_array)
    
    ## exclude psds with only NaN values
    psds = array([psd for psd in psd_array if not isnan(psd).all()])

    ## find overall minimum and maxium values
    max_value = max([max(sublist) for sublist in psd_array])
    min_value = min([min(sublist) for sublist in psd_array])
    print(min_value, max_value)

    ## define empty lists
    dist, dist_maximas, bins_maximas, bins_medians, stds, mads = [], [], [], [], [], []
    
    count = 0

    for h in range(len(psd_array[1])):
        
        psd = psd_array[:,h]
        
        hist, bin_edges = histogram(psd, bins=bins, range=(min_value, max_value), density=density);
                
        bin_mids = 0.5*(bin_edges[1:] + bin_edges[:-1])
        
        ## normalization
#         if  True:
#             hist = [val / len(psd_array[:,h]) for val in hist]
#             config['set_density'] = True

        ## check if density works
        DX = abs(max_value-min_value)/bins
        SUM = sum(hist)
        if str(SUM*DX) != "1.0":
            count+=1
        
        ## modify histogram with range increment
        hist = hist*DX
        
        ## append values to list
        dist.append(hist)
        stds.append(std(hist))
        dist_maximas.append(max(hist))
        bins_maximas.append(bin_mids[argmax(hist)])
        mads.append(mad(hist)) 
        
        ## compute median
        psd = psd[~(isnan(psd))]
        bins_medians.append(median(psd[psd != 0]))

    ## undo log conversion    
    output = {}
    output['dist'] = array(dist)
    output['bin_mids'] = 10**array(bin_mids)
    output['bins_maximas'] = 10**array(bins_maximas)
    output['stds'] = 10**array(stds)
    output['mads'] = 10**array(mads)
    output['bins_medians'] = 10**array(bins_medians)
    output['set_density'] = density
    output['total'] = psd_array.shape[0]
    
    if count > 0:
        print(f" {count} errors found for density computation!!!")
    
    return output

In [ ]:
# def __calculate_density(psd_array, bins=20, density=False):

#     from numpy import argmax, std, median, isnan, array, histogram

#     ## exclude psds with only NaN values
#     psds = array([psd for psd in psd_array if not isnan(psd).all()])

    
#     ## find overall minimum and maxium values
#     max_value = max([max(sublist) for sublist in psd_array])
#     min_value = min([min(sublist) for sublist in psd_array])
#     print(min_value, max_value)

#     ## define empty lists
#     dist, dist_maximas, bins_maximas, bins_medians, stds, mads = [], [], [], [], [], []
    
#     for h in range(len(psd_array[1])):
        
#         psd = psd_array[:,h]
        
#         hist, bin_edges = histogram(psd, bins=bins, range=(min_value, max_value), density=density);
        
#         bin_mids = 0.5*(bin_edges[1:] + bin_edges[:-1])
        
# #         if density:
# #             hist = [val / len(psd_array[:,h]) for val in hist]

#         ## check if density works
#         DX = abs(max_value-min_value)/bins
#         SUM = sum(hist)
#         if str(SUM*DX) != "1.0":
#             count+=1
        
#         ## modify histogram with range increment
#         hist = hist*DX
        
#         ## append values to list
#         dist.append(hist)
#         stds.append(std(hist))
#         dist_maximas.append(max(hist))
#         bins_maximas.append(bin_mids[argmax(hist)])
#         mads.append(mad(hist)) 
        
#         ## compute median
#         psd = psd[~(isnan(psd))]
#         bins_medians.append(median(psd[psd != 0]))

#     ## undo log conversion    
#     output = {}
#     output['dist'] = array(dist)
#     output['bin_mids'] = 10**array(bin_mids)
#     output['bins_maximas'] = 10**array(bins_maximas)
#     output['stds'] = 10**array(stds)
#     output['mads'] = 10**array(mads)
#     output['bins_medians'] = 10**array(bins_medians)
#     output['set_density'] = density
#     output['total'] = psd_array.shape[0]
    
#     if count > 0:
#         print(f" {count} errors found for density computation!!!")
    
    
#     return output

In [ ]:
output = __calculate_density(psds, bins=200, density=True)

In [ ]:
def __makeplot_PPSD(ff, output, config):

    from matplotlib import colors
    from numpy import ones, convolve

    
    def moving_average(x, w):
        return convolve(x, ones(w), 'same') / w

    
    bins, bins_medians, bins_maximas, dist = output['bin_mids'], output['bins_medians'], output['bins_maximas'], output['dist']
    
    y_axis = 10**bins
    x_axis = ff

    
    
    ## plotting
    fig = plt.figure(figsize=(13,6))

    font=14

    cmap = plt.cm.get_cmap('magma_r')
    cmap.set_under(color='white')

    im = plt.pcolor(x_axis, y_axis, dist.T, 
                    cmap=cmap,
                    vmin=1e-12, 
#                     norm=colors.LogNorm(), 
                   )

    fs = x_axis[2]-x_axis[1]
    
    bins_maximas_avg = moving_average(bins_maximas, 5)
    
    plt.plot(x_axis+fs/2, 10**array(bins_maximas_avg), color='black', alpha=0.5)
    plt.plot(x_axis+fs/2, 10**array(bins_medians), color='white', alpha=0.5)
    

    plt.xscale('log')
    plt.yscale('log')

#     plt.xlim(config['frequency_limits'][0], config['frequency_limits'][1])
#     plt.xlim(min(ff), max(ff))
    plt.xlim(0.005, 5)

    ## ROMY
#     plt.ylim(10**-24, 10**-16)

    ##RLAS
#     plt.ylim(10**-27, 10**-17)
    
    caxes = fig.add_axes()

    cb = plt.colorbar(im, cax=caxes)
    
    if output['set_density']:
        cb.set_label("Propability Density", fontsize=font)
    else:
        cb.set_label("Counts", fontsize=font)
    
    plt.ylabel(r"PPSD (rad$^2$/s$^2$/$Hz)$", fontsize=font)
    # plt.ylabel(f"PSD (dB rel. to {round(max_value,3)} V$^2$/Hz )",fontsize=font)
    plt.xlabel("Frequency (Hz)",fontsize=font)
    plt.title(f"PPSD {config['ring']} [{config['date1'].date} - {config['date2'].date}]; total = {output['total']}",fontsize=font+2)
    
    plt.tick_params(axis='both', labelsize=font-2)

    plt.show();
    return fig

In [ ]:
fig = __makeplot_PPSD(ff, output, config)

In [ ]:
output = __calculate_density(psds, bins=100, density=True)

In [ ]:
def __makeplot_PPSD(ff, output, config, mode=None):

    from matplotlib import colors
    from numpy import ones, convolve, array, log10, roll, sqrt

    bins, bins_medians, bins_maximas, dist = output['bin_mids'], output['bins_medians'], output['bins_maximas'], output['dist']
    
    if mode is "DB":
        y_axis = 10*log10(bins)
    elif mode is "PSD":
        y_axis = bins
    elif mode is "ASD":
        y_axis = sqrt(bins)
        
    x_axis = ff

    ## plotting
    fig, ax = plt.subplots(1,1,figsize=(13,6))

    font=14

    cmap = plt.cm.get_cmap('magma_r')
#     cmap = plt.cm.get_cmap('gray_r')
    cmap.set_under(color='white')

    im = plt.pcolor(x_axis, y_axis, dist.T, 
                    cmap=cmap,
                    vmin=1e-12, 
                    alpha=0.99,
                   )

    ## add Frequency Axis
    g = lambda x: 1/x
    ax2 = ax.secondary_xaxis("top", functions=(g,g))
    ax2.set_xlabel("Frequency in Hz", fontsize=font, labelpad=5)
    ax2.set_xticklabels(1/ff, fontsize=11)    
    
    
    if mode is "DB":
        plt.plot(x_axis, 10*log10(bins_medians), color='black', alpha=0.5, label="RNLM")
        plt.ylim(-200, -80)

    elif mode is "PSD":
        plt.plot(x_axis, bins_medians, color='black', alpha=0.5, label="RNLM")
        plt.yscale('log')
        plt.ylim(10**-20, 10**-13)
  
    elif mode is "ASD":  
        plt.plot(x_axis, sqrt(bins_medians), color='black', alpha=0.5, label="RNLM")
        plt.yscale('log')
        plt.ylim(10**-10, 2*10**-7)

    
#     plt.fill_betweenx(range(-270, -140), x1=0, x2=2.5, color="grey", alpha=0.4)
#     plt.fill_betweenx(range(-270, -140), x1=200, x2=max(ff), color="grey", alpha=0.4)

    ## modify x-axis
    plt.xscale('log')
    plt.xlim([min(x_axis), max(x_axis)])
#     plt.xlim(1, 300)
    
    ## add colorbar
    caxes = fig.add_axes()
    cb = plt.colorbar(im, cax=caxes, alpha=1.0)
    
    if output['set_density']:
        cb.set_label("Propability Density", fontsize=font)
    else:
        cb.set_label("Counts", fontsize=font)
        cb.set_label("Propability Density", fontsize=font)
    
    if mode is "DB":
#         plt.ylabel(r"PPSD in dB (rel. to 1 m$^4$/s$^2$/$Hz$)", fontsize=font)
        plt.ylabel(r"PPSD in dB (rel. to 1 rad$^2$/s$^2$/$Hz$)", fontsize=font)
    elif mode is "PSD":
#         plt.ylabel(r"PPSD in m$^4$/s$^2$/$Hz$", fontsize=font)
        plt.ylabel(r"PPSD in $\frac{rad^2}{s^2 Hz}$", fontsize=font)
    elif mode is "ASD":
        plt.ylabel(r"PASD in $\frac{rad}{s \sqrt{Hz}}$", fontsize=font)
        
    plt.grid(axis="both", which="both", ls="--", zorder=0)
    plt.legend()
    
    plt.xlabel("Period in seconds", fontsize=font)

    plt.tick_params(axis='both', labelsize=font-2)

    plt.show();
    return fig

In [ ]:
fig = __makeplot_PPSD(ff, output, config, mode="ASD")